## PtyPy moonflower example
#### scan model: BlockFull
#### engine: Difference Map (DM)

In [ ]:
from ptypy.core import Ptycho
from ptypy import utils as u

In [ ]:
# create parameter tree
p = u.Param()

In [ ]:
# set verbose level to interactive
p.verbose_level = "interactive"

In [ ]:
# set home path and io settings (no files saved)
p.io = u.Param()
p.io.rfile = None
p.io.autosave = u.Param(active=False)
p.io.autoplot = u.Param(active=False)
p.io.interaction = u.Param(active=False)

In [ ]:
# max 200 frames (128x128px) of diffraction data
p.scans = u.Param()
p.scans.MF = u.Param()
p.scans.MF.name = 'BlockFull'
p.scans.MF.data= u.Param()
p.scans.MF.data.name = 'MoonFlowerScan'
p.scans.MF.data.shape = 128
p.scans.MF.data.num_frames = 200
p.scans.MF.data.save = None
p.scans.MF.data.density = 0.2
p.scans.MF.data.photons = 1e8
p.scans.MF.data.psf = 0.

In [ ]:
# difference map reconstrucion engine
p.engines = u.Param()
p.engines.engine00 = u.Param()
p.engines.engine00.name = 'DM'
p.engines.engine00.numiter = 80

In [ ]:
# prepare and run
P = Ptycho(p,level=5)

## Plotting the results

In [ ]:
import ptypy.utils.plot_client as pc
fig = pc.figure_from_ptycho(P)